# Downloading and merging Wrocław city bike rentals and weather-related data using web-scraping

Before starting the data analysis process, our goal is to acquire the data, combine it, clean in, as well as possible transformations. We need data on Wrocław's city bike rentals. Fortunately, they are publicly available at https://www.wroclaw.pl/open-data/dataset/przejazdy-wroclawskiego-roweru-miejskiego-archiwalne. They cover the period from March 2019 to November 2021 - each CSV file covers one month. I downloaded the data immediately and moved it to the folder "raw-data". We will check if each file has the same structure - if not, we will unify it. After the corrections, we will merge all these files so that we have one - covering the rentals in this time frame.

In the analyses, I want to see how weather factors for Wroclaw correlate with bike rentals. This is an interesting question, as it will bring us closer to seeing how particular weather factors interfere with user behavior - both in the number of rentals and the length of travel time. Perhaps we can construct a machine learning model that predicts the average daily number of rentals based on weather factors? To address these issues, we need weather data. Unfortunately, these are not publicly available, and usually offered as HTML tables on websites.  

## Importing modules, setting parameters

In [37]:
from os import listdir

import pandas as pd
import polars as pl
import requests
from bs4 import BeautifulSoup

RAW_DATA_PATH = 'data/raw-data/'
CLEANED_DATA_PATH = 'data/cleaned-data/'

## Collecting and combining data

### Wrocław city bike rentals

Let's get started with city bikes! We create a parameter for the name of the final file (which is a combination of all monthly data files from 2019 to 2021) and define three functions to get only CSV files that are only related to rentals data and check if the file structure has a correct format (some files have no information about the duration of the loan - then we must calculate it by subtracting the loan date from the return date.

In [35]:
BIKES_FINAL_NAME = 'bikes_combined.csv'

In [3]:
def get_csv_only(path: str, start_str="", suffix=".csv") -> list:
    filenames = listdir(path)
    return [filename for filename in filenames if (filename.endswith(suffix) and (filename.startswith(start_str)))]

In [4]:
def is_wrong_struct(df: pl.DataFrame) -> bool:
    return False if "Czas trwania" in df.columns else True

In [70]:
def old_format_to_new(df: pl.DataFrame, col_names: list) -> pl.DataFrame:
    df.drop_in_place(df.columns[0])
    df.columns = col_names[:-1]   
    df = df.with_columns(
        [
            # ((pl.col("EndTime") - pl.col("StartTime")).dt.seconds() / 60)\
            ((pl.col("EndTime") - pl.col("StartTime")).dt.minutes())
            .cast(pl.Int64)
            .alias("Duration")
        ]
            )
    return df

We have 32 CSV files on Wrocław city bike rentals data which indicates 32 months (2 years and 8 months). There are exactly 32 months between April 2019 and November 2022. So we don't have any monthly gaps in the data.

In [69]:
bike_files = get_csv_only(path=RAW_DATA_PATH, start_str="historia")

bike_files

['historia_przejazdow_2019-04.csv',
 'historia_przejazdow_2019-05.csv',
 'historia_przejazdow_2019-06.csv',
 'historia_przejazdow_2019-07.csv',
 'historia_przejazdow_2019-08.csv',
 'historia_przejazdow_2019-09.csv',
 'historia_przejazdow_2019-10.csv',
 'historia_przejazdow_2019-11.csv',
 'historia_przejazdow_2019-12.csv',
 'historia_przejazdow_2020-01.csv',
 'historia_przejazdow_2020-02.csv',
 'historia_przejazdow_2020-03.csv',
 'historia_przejazdow_2020-04.csv',
 'historia_przejazdow_2020-05.csv',
 'historia_przejazdow_2020-06.csv',
 'historia_przejazdow_2020-07.csv',
 'historia_przejazdow_2020-08.csv',
 'historia_przejazdow_2020-09.csv',
 'historia_przejazdow_2020-10.csv',
 'historia_przejazdow_2020-11.csv',
 'historia_przejazdow_2020-12.csv',
 'historia_przejazdow_2021-01.csv',
 'historia_przejazdow_2021-02.csv',
 'historia_przejazdow_2021-03.csv',
 'historia_przejazdow_2021-04.csv',
 'historia_przejazdow_2021-05.csv',
 'historia_przejazdow_2021-06.csv',
 'historia_przejazdow_2021-0

In [71]:
col_names = ["UID", "BikeNumber", "StartTime", "EndTime", "RentalPlace", "ReturnPlace", "Duration"]
for file_name in bike_files:
    print(file_name)
    bikes_df = pl.read_csv(f"{RAW_DATA_PATH}{file_name}", parse_dates=True)
    if is_wrong_struct(bikes_df):
        bikes_df = old_format_to_new(bikes_df, col_names)
    else:
        bikes_df.columns = col_names
    bikes_df.write_csv(f"{CLEANED_DATA_PATH}{file_name}")

historia_przejazdow_2019-04.csv
historia_przejazdow_2019-05.csv
historia_przejazdow_2019-06.csv
historia_przejazdow_2019-07.csv
historia_przejazdow_2019-08.csv
historia_przejazdow_2019-09.csv
historia_przejazdow_2019-10.csv
historia_przejazdow_2019-11.csv
historia_przejazdow_2019-12.csv
historia_przejazdow_2020-01.csv
historia_przejazdow_2020-02.csv
historia_przejazdow_2020-03.csv
historia_przejazdow_2020-04.csv
historia_przejazdow_2020-05.csv
historia_przejazdow_2020-06.csv
historia_przejazdow_2020-07.csv
historia_przejazdow_2020-08.csv
historia_przejazdow_2020-09.csv
historia_przejazdow_2020-10.csv
historia_przejazdow_2020-11.csv
historia_przejazdow_2020-12.csv
historia_przejazdow_2021-01.csv
historia_przejazdow_2021-02.csv
historia_przejazdow_2021-03.csv
historia_przejazdow_2021-04.csv
historia_przejazdow_2021-05.csv
historia_przejazdow_2021-06.csv
historia_przejazdow_2021-07.csv
historia_przejazdow_2021-08.csv
historia_przejazdow_2021-09.csv
historia_przejazdow_2021-10.csv
historia

In [72]:
bikes_combined_df = pl.read_csv(f"{CLEANED_DATA_PATH}historia_przejazdow_*.csv")

In [74]:
bikes_combined_df.head()

UID,BikeNumber,StartTime,EndTime,RentalPlace,ReturnPlace,Duration
i64,i64,str,str,str,str,i64
66572449,57768,"""2019-03-31T19:12:12.000000000""","""2019-04-01T00:01:03.000000000""","""Wróblewskiego (Teki)""","""Wróblewskiego (Teki)""",288
66582502,57555,"""2019-03-31T22:51:46.000000000""","""2019-04-01T00:02:27.000000000""","""Drobnera / Dubois""","""Poza stacją""",70
66583423,57122,"""2019-03-31T23:47:28.000000000""","""2019-04-01T00:03:28.000000000""","""Zachodnia / Poznańska""","""Plac Świętego Macieja / Trzebnicka""",16
66583399,57414,"""2019-03-31T23:45:36.000000000""","""2019-04-01T00:03:48.000000000""","""Plac Legionów""","""Plac Grunwaldzki / Polaka""",18
66583594,57448,"""2019-04-01T00:01:12.000000000""","""2019-04-01T00:05:38.000000000""","""Wita Stwosza / Szewska""","""Drobnera / Plac Bema""",4


In [75]:
bikes_combined_df.write_csv(f"{CLEANED_DATA_PATH}{BIKES_FINAL_NAME}")

### Weather

In [52]:
WEATHER_FINAL_NAME = 'weather.csv'
PAGE_URL = "https://meteomodel.pl/dane/historyczne-dane-pomiarowe/"
DATE_START = "2019-01-01"
DATE_END = "2022-12-31"
DATE_RANGE = pd.date_range(DATE_START, DATE_END, freq="100D")

DATE_RANGE

DatetimeIndex(['2019-01-01', '2019-04-11', '2019-07-20', '2019-10-28',
               '2020-02-05', '2020-05-15', '2020-08-23', '2020-12-01',
               '2021-03-11', '2021-06-19', '2021-09-27', '2022-01-05',
               '2022-04-15', '2022-07-24', '2022-11-01'],
              dtype='datetime64[ns]', freq='100D')

In [47]:
def scrap_weather_data(page_url: str, date_range):
    df_list = []
    for date in DATE_RANGE:
        print(date)
        day_url = f'''
        {PAGE_URL}?data={date}&rodzaj=st&imgwid=351160424&dni=100&ord=asc
        '''
        page_day = requests.get(day_url)
        soup_day = BeautifulSoup(page_day.content, "html.parser")
        table = soup_day.find("table", id="tablepl")
        table_rows = table.find_all("tr")
        for row in table_rows[2:]:
            row_data = row.find_all("td")
            date = row_data[0].find("a").text.strip()
            mean_temp = row_data[0].text.split()[1].strip()
            row_data_cleaned = [date, mean_temp] + [td.text.strip().replace("-", "") for td in row_data[2:]]
            df_list.append(row_data_cleaned)
    
    weather_df = pd.DataFrame(df_list, columns=["Date", "TMean", "TMax", "TMin", "TMinLand", "PrecMean", "V",
                                                "CloudMean", "WindMean", "WindMean10", "WindMean15", "PressSea", "PressStation",
                                                "PrecSum", "PrecDay", "PrecNight", "SnowHeight", "SnowWater", "Sun", "RainTime",
                                                "SnowTime", "RainSnowTime", "FogTime", "DewTime", "FrostTime", "StormTime"])
    return weather_df

In [48]:
weather_df = scrap_weather_data(page_url=PAGE_URL, date_range=DATE_RANGE)

2019-01-01 00:00:00
2019-04-11 00:00:00
2019-07-20 00:00:00
2019-10-28 00:00:00
2020-02-05 00:00:00
2020-05-15 00:00:00
2020-08-23 00:00:00
2020-12-01 00:00:00
2021-03-11 00:00:00
2021-06-19 00:00:00
2021-09-27 00:00:00
2022-01-05 00:00:00
2022-04-15 00:00:00
2022-07-24 00:00:00
2022-11-01 00:00:00


In [50]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1346 entries, 0 to 1345
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          1346 non-null   object
 1   TMean         1346 non-null   object
 2   TMax          1346 non-null   object
 3   TMin          1346 non-null   object
 4   TMinLand      1346 non-null   object
 5   PrecMean      1346 non-null   object
 6   V             1346 non-null   object
 7   CloudMean     1346 non-null   object
 8   WindMean      1346 non-null   object
 9   WindMean10    1346 non-null   object
 10  WindMean15    1346 non-null   object
 11  PressSea      1346 non-null   object
 12  PressStation  1346 non-null   object
 13  PrecSum       1346 non-null   object
 14  PrecDay       1346 non-null   object
 15  PrecNight     1346 non-null   object
 16  SnowHeight    1346 non-null   object
 17  SnowWater     1346 non-null   object
 18  Sun           1346 non-null   object
 19  RainTi

In [51]:
weather_df.head()

,Date,TMean,TMax,TMin,TMinLand,PrecMean,V,CloudMean,WindMean,WindMean10,...,SnowHeight,SnowWater,Sun,RainTime,SnowTime,RainSnowTime,FogTime,DewTime,FrostTime,StormTime
0,2018-09-24,9.0,14.2,7.7,6.3,76.4,8.7,6.8,5.6,0.0,...,,0.0,5.2,3.8,,,,,,
1,2018-09-25,8.3,13.4,4.8,3.9,73.0,7.8,3.4,3.4,0.0,...,,0.0,8.3,0.2,,,,5.0,,
2,2018-09-26,9.7,16.3,0.4,1.3,58.3,6.5,5.5,3.1,0.0,...,,0.0,8.1,,,,,2.0,4.0,
3,2018-09-27,15.6,20.8,11.0,8.7,61.6,11.0,5.6,3.9,0.0,...,,0.0,3.8,,,,,3.0,,
4,2018-09-28,13.1,19.4,7.3,5.0,70.0,10.5,3.8,1.9,0.0,...,,0.0,6.8,0.4,,,,7.5,,


In [76]:
weather_df[weather_df['Date'].between('2019-04-01', '2021-11-31')]

,Date,TMean,TMax,TMin,TMinLand,PrecMean,V,CloudMean,WindMean,WindMean10,...,SnowHeight,SnowWater,Sun,RainTime,SnowTime,RainSnowTime,FogTime,DewTime,FrostTime,StormTime
189,2019-04-01,6.0,11.3,2.0,4.8,47.4,4.2,0.8,4.3,0.0,...,,0.0,12.3,,,,,,4.0,
190,2019-04-02,8.7,16.5,0.9,0.9,45.9,5.1,2.1,6.4,0.5,...,,0.0,11.7,,,,,,,
191,2019-04-03,13.0,20.9,4.9,1.9,47.8,6.9,5.9,6.4,4.1,...,,0.0,9.5,,,,,4.0,,
192,2019-04-04,15.3,21.0,9.3,6.6,43.3,7.4,2.6,7.9,7.5,...,,0.0,10.9,,,,,,,
193,2019-04-05,16.0,21.9,7.8,3.5,45.5,8.0,4.1,4.1,0.0,...,,0.0,8.7,,,,,3.0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1159,2021-11-26,2.5,3.7,2.0,1.7,82.5,6.0,8.0,1.5,0.0,...,,0.0,0.0,3.6,4.8,0.4,,,,
1160,2021-11-27,1.6,4.5,0.3,1.9,87.5,6.0,5.9,2.8,0.0,...,,0.0,5.8,,0.2,,,,6.0,
1161,2021-11-28,-0.1,2.4,2.5,4.8,95.3,5.8,7.5,1.5,0.0,...,,0.0,0.0,,11.9,2.0,,,9.0,
1162,2021-11-29,0.8,1.7,0.7,0.8,91.8,6.0,7.6,4.0,0.0,...,6,1.8,0.0,,6.4,1.1,,,,


In [53]:
weather_df.to_csv(f'{CLEANED_DATA_PATH}{WEATHER_FINAL_NAME}', index=False)

### PM

## Data cleaning, feature engineering

### Wrocław city bike rentals

### Weather

### PM

## Final steps, putting everything together